# Projeto Final

Você fazem parte do time de Data Science e Analytics da Popolishoshop e receberam uma base de dados contendo as informações sobre a última Black Friday que ocorreu. O time de negócio solicitou para vocês um relatório, onde especificaram algumas informações e também um estudo para que vocês respondessem utilizando as bases fornecidas.

Para esse desafio, vamos trabalhar com o data set [Black Friday](https://www.kaggle.com/sdolezel/black-friday), que reúne dados sobre transações de compras em uma loja de varejo. Esse dataset está quebrado em diferentes arquivos e é sua função entender como cada um se relaciona com o outro.

Vamos utilizá-lo para praticar a exploração utilizando pandas.

Na tabela a seguir podemos ver os nomes das colunas e as descrições dos campos.

| Coluna                 | Descrição                                                 |
|------------------------|-----------------------------------------------------------|
| User_ID                | ID do usuário                                             |
| Product_ID             | ID do produto                                             |
| Gender                 | Sexo do usuário                                           |
| Age                    | Ano em intervalos                                         |
| Occupation             | Ocupação (mascarada)                                      |
| City_Category          | Categoria da cidade (A, B, C)                             |
| StayInCurrentCityYears | Número de anos de permanência na cidade atual             |
| Marital_Status         | Estado civil                                              |
| ProductCategory1       | Categoria do produto (Mascarada)                          |
| ProductCategory2       | Categoria que o produto pode pertencer também (Mascarada) |
| ProductCategory3       | Categoria que o produto pode pertencer também (Mascarada) |
| Purchase               | Valor da compra                                           | 

Todo o código desenvolvido deve ser pensado para ser reutilizado. A avaliação se dará executando todo o notebook com outra tabela, de mesmas colunas. Sendo assim, pensem na qualidade e reprodução do código.

## _Set up_ da análise

Faça a leitura das três bases fornecidas e junte-as em um único DataFrame.

In [167]:
import pandas as pd
import numpy as np

In [169]:
df_product_info = pd.read_csv("'./data/product_info.csv", delimiter=';')
df_purchase = pd.read_csv("'./data/purchase.csv")
df_user_prof = pd.read_csv("'./data/user_profile.csv", delimiter='|')

In [170]:
df_shop = pd.merge(df_user_prof, df_purchase, how='inner', on='User_ID')
df_shop = pd.merge(df_shop, df_product_info, how='inner', on='Product_ID')
df_shop.head()

,User_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_ID,Purchase,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,F,0-17,10,A,2,0,P00069042,8370,3,NaN,NaN
1,1000017,M,51-55,1,C,0,0,P00069042,13591,3,NaN,NaN
2,1000023,M,36-45,0,B,3,1,P00069042,11022,3,NaN,NaN
3,1000030,F,36-45,7,C,3,0,P00069042,8036,3,NaN,NaN
4,1000073,M,18-25,4,A,4+,0,P00069042,10885,3,NaN,NaN


## Questão 1

Quantas observações e quantas colunas há no dataset completo (todas as bases juntas)? Responda no formato de uma tuple `(n_observacoes, n_colunas)`.

In [171]:
#O método shape retorna uma tupla com os dois valores, nº de linhas e nº de colunas.

df_shop.shape

(537577, 12)

## Questão 2

Há quantas mulheres com idade entre 26 e 35 anos no dataset? Responda como um único escalar.

In [172]:
#Realizei dois filtros juntos pra retornar um único valor.

mulheres = (df_shop['Gender'] == 'F') & (df_shop['Age'] == '26-35')
mulheres.sum()

49348

## Questão 3

Quantos usuários únicos há no dataset? Responda como um único escalar.

In [173]:
df_shop['User_ID'].nunique(dropna=True)

5891

## Questão 4

Qual porcentagem dos registros (percentual de linhas) possui ao menos um valor null (`None`, `ǸaN` etc)? Responda como um único escalar entre 0 e 1.

In [174]:
#Realizei primeiro uma subtração entre o número de linhas do df inteiro 
#pelo número de linhas que não possuem missings, após isso dividi pelo número
#de linhas totais novamente, obtendo um único escalar.

round((len(df_shop) - len(df_shop.dropna()))/len(df_shop),5)

0.69441

## Questão 5

Quantos valores null existem na variável (coluna) com o maior número de null? Responda como um único escalar.

In [175]:
#Do mesmo modo que o exercício anterior, fiz em uma mesma linha a função de soma dos valores NULOS,
#e em seguida, a função para obter o valor máximo da lista.

df_shop.isnull().sum().max()

373299

## Questão 6

Qual o valor mais frequente (sem contar nulls) em `Product_Category_3`? Responda como um único escalar.

In [176]:
#A função 'value_counts' mostra a lista de valores que existem na coluna 'PC_3'.
#Quando realizo a função 'idxmax', ela me mostra o índex do valor mais repetido na coluna,
#ao invés da quantidade de vezes que foi inserido.

df_shop['Product_Category_3'].value_counts().idxmax()

16.0

## Questão 7

Podemos afirmar que se uma observação é null em `Product_Category_2` ela também o é em `Product_Category_3`? Responda com um bool (`True`, `False`).

In [177]:
#Decidi criar uma função que verifica o elemento de mesmo índice da coluna 'PC2' com a 'PC3'

def verifica_nulos(x, y):
    
    for i in range (len(x)):
        
        if x[i] == np.NaN and y[i] != np.NaN:
            
            return False
        
    return True

In [47]:
verifica_nulos(df_shop['Product_Category_2'], df_shop['Product_Category_3'])

True

## Questão 8

Qual o ID do usuário que mais gastou na Black Friday?

In [178]:
#A coluna Purchase indica o valor dos gastos, então procurei filtrar
#através de um 'groupby', somando os valores gastos por cada ID,
#depois realizando a função 'idxmax' mostrando o índice MÁXIMO do dataframe criado.

df_shop.groupby(['User_ID'])['Purchase'].sum().idxmax()

1004277

## Questão 9

Qual grupo (homens ou mulheres) mais gastou na Black Friday?

In [179]:
#Da mesma forma que a questão anterior, realizamos um agrupamento
#dos dados por um 'groupby' e somando valor de compra de cada gênero.

df_shop.groupby(['Gender'])['Purchase'].sum()

Gender
F    1164624021
M    3853044357
Name: Purchase, dtype: int64

In [180]:
#Chegamos a conclusão de que as MULHERES foram o grupo que mais consumiu na Black Friday.

## Questão 10

Faça uma nova tabela com a categoria mais comprada por cada cliente.

Obs: se ele comprou um produto que possuir valores nas três colunas de categorias, então deve-se considerar todas as categorias.

## Questão 11

Normalize a coluna Purchase. A fórmula de normalização é:


$$x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}} $$

In [19]:
x_max = df_shop['Purchase'].max()
x_min = df_shop['Purchase'].min()

df_shop['Purchase'] = df_shop['Purchase'].apply(lambda x: (x - x_min)/(x_max - x_min))
df_shop['Purchase'].head()

0    0.344255
1    0.563846
2    0.455796
3    0.330207
4    0.450034
Name: Purchase, dtype: float64

## Questão 12
O estado civil influencia no valor gasto e na categoria de produto comprada? Mostre!

Se eu quisesse vender mais produtos da categoria 14, deveria investir em propagandas para qual estado civil?

## Questão 13
Quais as variáveis que mais impactaram no valor da compra? Como você chegou a essa conclusão?